- Preciso filtrar orders fulfillment
- Preciso filtrar orders aprovadas
- Preciso filtrar 'date_approved' em 'payments'. Se não houver desconsiderar

------

em cada resultado da lista de json existe um "shipping" com "id":
- preciso passar este id em: https://api.mercadolibre.com/shipments/{ship_id}
- procurar por 'fulfillment' em "logistic_type"
- se for fullfilment devo acrescentar dados ao df que calcula a qtd de produtos vendido

Em fulfillment aprovadas:
- Preciso saber a quantidade de vendas por id em determinado período (mesmo período da consulta ao estoque fulfillment, sempre findando no dia anterior a hoje)

In [122]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(ACCESS_TOKEN)

APP_USR-6628000663056293-111015-baf0289ea26e906c5a3d94b0671ddaa4-233632476


### Verificando dados da conta

In [3]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4519,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1505,"completed":32625,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

## Pegando dados de Vendas

A consulta a API do ML é feita com filtros para order.status = 'paid' e datas pré-determinadas

### Consultando API

In [147]:
# date_from = '2023-10-10'
date_from = '2023-11-09'
date_to = '2023-11-10'

In [148]:
# URL base da API
base_url = "https://api.mercadolibre.com/orders/search"

# Parâmetros iniciais
params = {
    'seller': '233632476',
    # 'order.status': 'paid',
    'order.date_created.from': f'{date_from}T00:00:00.000-00:00',
    'order.date_created.to': f'{date_to}T00:00:00.000-00:00',
    'limit': 50,
    'offset': 0
}

# Token de autenticação
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

json_list = []

In [149]:
counter = 0

while True:
    response = requests.get(base_url, params=params, headers=headers)
    data = response.json()

    
    if 'results' in data:
        json_list.extend(data['results'])
    else:
        break
    
    # Verifique se há mais páginas
    if 'paging' in data:
        total_paging = data['paging'].get('total')
        if total_paging is None:
            break

        total_pages = math.ceil(total_paging / params['limit'])
        print(f'Total esperado de páginas: {counter}/{total_pages}')
        print(f'Offset atual: {params["offset"]}')

        counter += 1
        if params['offset'] >= total_paging:
            break

        params['offset'] += params['limit']
    else:
        break

print(f'Total de dados coletados: {len(json_list)}')

Total esperado de páginas: 0/4
Offset atual: 0
Total esperado de páginas: 1/4
Offset atual: 50
Total esperado de páginas: 2/4
Offset atual: 100
Total esperado de páginas: 3/4
Offset atual: 150
Total esperado de páginas: 4/4
Offset atual: 200
Total de dados coletados: 180


In [150]:
len(json_list)

180

In [151]:
json_list[0]

{'payments': [{'reason': 'Lâmina De Serra Manual Bimetal 12  Com 18 Dentes Por Polegada - Bs1218 Starrett',
   'status_code': None,
   'total_paid_amount': 104.04,
   'operation_type': 'regular_payment',
   'transaction_amount': 104.04,
   'transaction_amount_refunded': 0,
   'date_approved': '2023-11-08T20:09:15.000-04:00',
   'collector': {'id': 233632476},
   'coupon_id': None,
   'installments': 1,
   'authorization_code': None,
   'taxes_amount': 0,
   'id': 66491818765,
   'date_last_modified': '2023-11-08T20:10:50.000-04:00',
   'coupon_amount': 0,
   'available_actions': ['refund'],
   'shipping_cost': 0,
   'installment_amount': None,
   'date_created': '2023-11-08T20:08:41.000-04:00',
   'activation_uri': None,
   'overpaid_amount': 0,
   'card_id': None,
   'status_detail': 'accredited',
   'issuer_id': '12501',
   'payment_method_id': 'pix',
   'payment_type': 'bank_transfer',
   'deferred_period': None,
   'atm_transfer_reference': {'transaction_id': None, 'company_id': '1

### Pegando linhas onde existe 'date_approved'

In [152]:
meta_cols = ['date_closed', 'pack_id', 'shipping', 'order_items']

df_payments = json_normalize(json_list, record_path=['payments'], meta=meta_cols)
df_payments.columns


Index(['reason', 'status_code', 'total_paid_amount', 'operation_type',
       'transaction_amount', 'transaction_amount_refunded', 'date_approved',
       'coupon_id', 'installments', 'authorization_code', 'taxes_amount', 'id',
       'date_last_modified', 'coupon_amount', 'available_actions',
       'shipping_cost', 'installment_amount', 'date_created', 'activation_uri',
       'overpaid_amount', 'card_id', 'status_detail', 'issuer_id',
       'payment_method_id', 'payment_type', 'deferred_period', 'site_id',
       'payer_id', 'order_id', 'currency_id', 'status', 'transaction_order_id',
       'collector.id', 'atm_transfer_reference.transaction_id',
       'atm_transfer_reference.company_id', 'date_closed', 'pack_id',
       'shipping', 'order_items'],
      dtype='object')

In [153]:
df_payments.head(1)

,reason,status_code,total_paid_amount,operation_type,transaction_amount,transaction_amount_refunded,date_approved,coupon_id,installments,authorization_code,...,currency_id,status,transaction_order_id,collector.id,atm_transfer_reference.transaction_id,atm_transfer_reference.company_id,date_closed,pack_id,shipping,order_items
0,Lâmina De Serra Manual Bimetal 12 Com 18 Dent...,None,104.04,regular_payment,104.04,0.0,2023-11-08T20:09:15.000-04:00,None,1,None,...,BRL,approved,None,233632476,None,1,2023-11-08T20:09:42.000-04:00,None,{'id': 42780216080},"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ..."


In [154]:
cols = ['date_approved','status','shipping','order_items']
df_payments = df_payments[cols]

col = {'status':'payment_status'}
df_payments.rename(columns=col, inplace= True)

df_payments.head(1)

,date_approved,payment_status,shipping,order_items
0,2023-11-08T20:09:15.000-04:00,approved,{'id': 42780216080},"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ..."


In [155]:
print(df_payments.shape)
df_payments = df_payments.dropna(subset=['date_approved'])
print(df_payments.shape)

(187, 4)
(180, 4)


In [156]:
df_payments.head(3)

,date_approved,payment_status,shipping,order_items
0,2023-11-08T20:09:15.000-04:00,approved,{'id': 42780216080},"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ..."
2,2023-11-08T20:22:47.000-04:00,approved,{'id': 42780148489},"[{'item': {'id': 'MLB3427769549', 'title': 'Pa..."
3,2023-11-08T20:24:44.000-04:00,refunded,{'id': 42780276438},"[{'item': {'id': 'MLB3473725713', 'title': 'Su..."


#### Pegando 'shipping.id' 

Para verificar se uma ordem do ML foi do modo Full devo encontrar o shipping.id

In [157]:
df_payments['shipping_id'] = df_payments['shipping'].apply(lambda x: x['id'])
df_payments = df_payments.drop('shipping', axis=1)
df_payments.head(1)

,date_approved,payment_status,order_items,shipping_id
0,2023-11-08T20:09:15.000-04:00,approved,"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ...",42780216080


### Pegando dados de produtos

In [158]:
df_orders = json_normalize(json_list, record_path=['order_items'], meta=['date_closed', 'pack_id', 'status', 'shipping'])

## pd.set_option('display.max_columns', None)
df_orders.columns

Index(['quantity', 'unit_price', 'full_unit_price', 'currency_id',
       'manufacturing_days', 'picked_quantity', 'sale_fee', 'listing_type_id',
       'base_exchange_rate', 'base_currency_id', 'bundle', 'element_id',
       'item.id', 'item.title', 'item.category_id', 'item.variation_id',
       'item.seller_custom_field', 'item.global_price', 'item.net_weight',
       'item.variation_attributes', 'item.warranty', 'item.condition',
       'item.seller_sku', 'requested_quantity.measure',
       'requested_quantity.value', 'date_closed', 'pack_id', 'status',
       'shipping'],
      dtype='object')

In [159]:
cols = ['quantity', 'item.id', 'item.title', 'item.category_id', 'item.variation_id',
       'item.seller_sku', 'date_closed', 'pack_id', 'status',
       'shipping']

df_orders = df_orders[cols]
df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping
0,6,MLB3470056711,Lâmina De Serra Manual Bimetal 12 Com 18 Dent...,MLB271774,NaN,C4BS1218,2023-11-08T20:09:42.000-04:00,None,paid,{'id': 42780216080}


#### Pegando 'shipping.id' 

Para verificar se uma ordem do ML foi do modo Full devo encontrar o shipping.id

In [160]:
df_orders['shipping_id'] = df_orders['shipping'].apply(lambda x: x['id'])
df_orders = df_orders.drop('shipping', axis=1)
df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id
0,6,MLB3470056711,Lâmina De Serra Manual Bimetal 12 Com 18 Dent...,MLB271774,NaN,C4BS1218,2023-11-08T20:09:42.000-04:00,None,paid,42780216080


In [161]:
print(df_payments.shape)
print(df_orders.shape)

(180, 4)
(180, 10)


## Combinando df_orders e df_payments para calcular produtos

In [162]:
df_orders.head(1)


,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id
0,6,MLB3470056711,Lâmina De Serra Manual Bimetal 12 Com 18 Dent...,MLB271774,NaN,C4BS1218,2023-11-08T20:09:42.000-04:00,None,paid,42780216080


In [163]:
df_payments.head(1)

,date_approved,payment_status,order_items,shipping_id
0,2023-11-08T20:09:15.000-04:00,approved,"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ...",42780216080


In [164]:
print(df_orders.shape)
print(df_payments.shape)

(180, 10)
(180, 4)


In [165]:
# df_resultado = pd.merge(df_orders, df_payments, on='shipping_id', how='left')
# df_resultado

In [166]:
mascara_orders = df_orders['shipping_id'].isin(df_payments['shipping_id'])
novo_df_orders = df_orders[mascara_orders]

In [167]:
novo_df_orders = novo_df_orders.merge(df_payments, on='shipping_id', how='left')
novo_df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status,order_items
0,6,MLB3470056711,Lâmina De Serra Manual Bimetal 12 Com 18 Dent...,MLB271774,NaN,C4BS1218,2023-11-08T20:09:42.000-04:00,None,paid,42780216080,2023-11-08T20:09:15.000-04:00,approved,"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ..."


In [168]:
novo_df_orders.shape

(188, 13)

In [169]:
novo_df_orders.columns

Index(['quantity', 'item.id', 'item.title', 'item.category_id',
       'item.variation_id', 'item.seller_sku', 'date_closed', 'pack_id',
       'status', 'shipping_id', 'date_approved', 'payment_status',
       'order_items'],
      dtype='object')

In [170]:
novo_df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status,order_items
0,6,MLB3470056711,Lâmina De Serra Manual Bimetal 12 Com 18 Dent...,MLB271774,NaN,C4BS1218,2023-11-08T20:09:42.000-04:00,None,paid,42780216080,2023-11-08T20:09:15.000-04:00,approved,"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ..."


Pode haver mais de um produto por envio. Logo, shipping_id se repete no DF. Vamos remover as duplicatas para consultar o endpoint de envios 

In [171]:
df_orders['shipping_id'].value_counts()

shipping_id
42783106137    3
42783914396    2
42782855675    1
42782848528    1
42782737967    1
              ..
42781875054    1
42781781949    1
42781964356    1
42781873919    1
42783800806    1
Name: count, Length: 177, dtype: int64

In [172]:
x = 42686145067
y = df_orders[df_orders['shipping_id'] == x]

# pd.set_option('display.max_colwidth', None)
y

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id


## Consultando shipping_id

Devo verificar o tipo de envio, se for fulfillment devo sinalizar para futuramente filtrar o df criado no passo anterior

https://developers.mercadolivre.com.br/pt_br/gerenciamento-de-envios

```
curl -X OPTION -H 'Authorization: Bearer $ACCESS_TOKEN' https://api.mercadolibre.com/shipments/$SHIPMENT_ID
```

In [173]:
# pegando valores unicos para consula futura

uniq_shipping_id = df_orders['shipping_id'].unique()


print(len(uniq_shipping_id))
print(df_orders.shape)
print(type(uniq_shipping_id))

177
(180, 10)
<class 'numpy.ndarray'>


In [174]:
json_shipments_list = []
success_count = 0
error_count = 0
counter = 0

for shipping_id in uniq_shipping_id:
    url = f"https://api.mercadolibre.com/shipments/{shipping_id}"

    payload = {}
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}'
    }

    try:
        print(f'Loop nº {counter}/{len(uniq_shipping_id)}: shipping_id = {shipping_id}')
        response = requests.request("GET", url, headers=headers, data=payload)
        response.raise_for_status()  # Verifica se a resposta tem um status de sucesso (código 2xx)

        json_shipments_list.append(response.json())
        success_count += 1
    except requests.exceptions.RequestException as e:
        # Lidar com erros de solicitação (por exemplo, conexão perdida, timeout)
        error_count += 1
        print(f"Erro na solicitação para shipping_id {shipping_id}: {str(e)}")
    counter +=1

print(f"Solicitações bem-sucedidas: {success_count}")
print(f"Erros de solicitação: {error_count}")


Loop nº 0/177: shipping_id = 42780216080
Loop nº 1/177: shipping_id = 42780148489
Loop nº 2/177: shipping_id = 42780276438
Loop nº 3/177: shipping_id = 42780312778
Loop nº 4/177: shipping_id = 42780347140
Loop nº 5/177: shipping_id = 42780249069
Loop nº 6/177: shipping_id = 42780415422
Loop nº 7/177: shipping_id = 42780407714
Loop nº 8/177: shipping_id = 42780282481
Loop nº 9/177: shipping_id = 42780316069
Loop nº 10/177: shipping_id = 42780352733
Loop nº 11/177: shipping_id = 42780320601
Loop nº 12/177: shipping_id = 42780363563
Loop nº 13/177: shipping_id = 42780386625
Loop nº 14/177: shipping_id = 42780549752
Loop nº 15/177: shipping_id = 42780449463
Loop nº 16/177: shipping_id = 42780563194
Loop nº 17/177: shipping_id = 42780499827
Loop nº 18/177: shipping_id = 42780651000
Loop nº 19/177: shipping_id = 42780604597
Loop nº 20/177: shipping_id = 42780615113
Loop nº 21/177: shipping_id = 42780755880
Loop nº 22/177: shipping_id = 42780666439
Loop nº 23/177: shipping_id = 42780805090
Lo

In [175]:
len(json_shipments_list)
print(type(json_shipments_list))

<class 'list'>


In [176]:
json_shipments_list

[{'substatus_history': [{'date': '2023-11-08T20:09:47.000-04:00',
    'substatus': 'invoice_pending',
    'status': 'ready_to_ship'},
   {'date': '2023-11-09T07:44:58.927-04:00',
    'substatus': 'waiting_for_carrier_authorization',
    'status': 'ready_to_ship'},
   {'date': '2023-11-09T07:45:00.591-04:00',
    'substatus': 'ready_to_print',
    'status': 'ready_to_ship'},
   {'date': '2023-11-10T15:10:39.000-04:00',
    'substatus': 'picked_up',
    'status': 'ready_to_ship'}],
  'snapshot_packing': {'snapshot_id': '91e99346-388d-4609-ab07-267b6ee8bbdb',
   'pack_hash': '1'},
  'receiver_id': 374323266,
  'base_cost': 47.6,
  'status_history': {'date_shipped': None,
   'date_returned': None,
   'date_delivered': None,
   'date_first_visit': None,
   'date_not_delivered': None,
   'date_cancelled': None,
   'date_handling': '2023-11-08T20:09:47.000-04:00',
   'date_ready_to_ship': '2023-11-08T20:09:47.000-04:00'},
  'type': 'forward',
  'return_details': None,
  'sender_id': 233632476

In [177]:
df = pd.DataFrame(json_shipments_list)

# df.head(1)
df.columns


Index(['substatus_history', 'snapshot_packing', 'receiver_id', 'base_cost',
       'status_history', 'type', 'return_details', 'sender_id', 'mode',
       'order_cost', 'priority_class', 'service_id', 'shipping_items',
       'tracking_number', 'cost_components', 'id', 'tracking_method',
       'last_updated', 'items_types', 'comments', 'substatus', 'date_created',
       'date_first_printed', 'created_by', 'application_id', 'shipping_option',
       'tags', 'sender_address', 'sibling', 'return_tracking_number',
       'site_id', 'carrier_info', 'market_place', 'receiver_address',
       'customer_id', 'order_id', 'quotation', 'status', 'logistic_type'],
      dtype='object')

In [178]:
df.head(1)

,substatus_history,snapshot_packing,receiver_id,base_cost,status_history,type,return_details,sender_id,mode,order_cost,...,return_tracking_number,site_id,carrier_info,market_place,receiver_address,customer_id,order_id,quotation,status,logistic_type
0,"[{'date': '2023-11-08T20:09:47.000-04:00', 'su...",{'snapshot_id': '91e99346-388d-4609-ab07-267b6...,374323266,47.6,"{'date_shipped': None, 'date_returned': None, ...",forward,None,233632476,me2,104.04,...,None,MLB,None,MELI,"{'country': {'id': 'BR', 'name': 'Brasil'}, 'a...",None,2000006849940598,None,ready_to_ship,cross_docking


In [179]:
print(df['logistic_type'].value_counts())

logistic_type
fulfillment      141
cross_docking     27
self_service       9
Name: count, dtype: int64


In [180]:
df_orders = novo_df_orders.copy()
# Buscando envios do Full

df_fulfillment_ship = df[['id', 'logistic_type']]

df_fulfillment_ship = df_fulfillment_ship[df_fulfillment_ship['logistic_type'] == 'fulfillment']

print(df_orders.shape)
print(df_fulfillment_ship.shape)

print(df_fulfillment_ship['logistic_type'].value_counts())

(188, 13)
(141, 2)
logistic_type
fulfillment    141
Name: count, dtype: int64


In [181]:
df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status,order_items
0,6,MLB3470056711,Lâmina De Serra Manual Bimetal 12 Com 18 Dent...,MLB271774,NaN,C4BS1218,2023-11-08T20:09:42.000-04:00,None,paid,42780216080,2023-11-08T20:09:15.000-04:00,approved,"[{'item': {'id': 'MLB3470056711', 'title': 'Lâ..."


In [182]:
df_fulfillment_ship.head(1)

,id,logistic_type
1,42780148489,fulfillment


In [183]:
# Unindo os DFs. Retorna apenas as linhas em df_orders onde shipping_id == id
df_res_fulfillment = pd.merge(df_orders, df_fulfillment_ship, left_on='shipping_id', right_on='id', how='inner')

df_res_fulfillment.shape

(141, 15)

In [184]:
df_res_fulfillment.columns

Index(['quantity', 'item.id', 'item.title', 'item.category_id',
       'item.variation_id', 'item.seller_sku', 'date_closed', 'pack_id',
       'status', 'shipping_id', 'date_approved', 'payment_status',
       'order_items', 'id', 'logistic_type'],
      dtype='object')

In [185]:
cols = ['quantity','item.id','item.seller_sku','date_closed', 'date_approved', 'pack_id', 'status', 'shipping_id', 'logistic_type']
df_order_res = df_res_fulfillment[cols]
df_order_res

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type
0,1,MLB3427769549,FULLIZZO6480,2023-11-08T20:22:47.000-04:00,2023-11-08T20:22:47.000-04:00,None,paid,42780148489,fulfillment
1,3,MLB3473725713,FULLHERCULESGSP39WB,2023-11-08T20:24:44.000-04:00,2023-11-08T20:24:44.000-04:00,2000005018790075,cancelled,42780276438,fulfillment
2,1,MLB1453957901,FULLKALANI15414,2023-11-08T20:33:14.000-04:00,2023-11-08T20:33:14.000-04:00,None,paid,42780312778,fulfillment
3,1,MLB1971185030,FULLBRANCO3440,2023-11-08T20:42:51.000-04:00,2023-11-08T20:42:50.000-04:00,2000005018869963,paid,42780347140,fulfillment
4,1,MLB3780803056,FULLBASSOCLA09,2023-11-08T20:46:36.000-04:00,2023-11-08T20:46:35.000-04:00,2000005018888075,paid,42780249069,fulfillment
...,...,...,...,...,...,...,...,...,...
136,1,MLB1992889189,FULLLIVERPOOL3NI7AM,2023-11-09T20:18:41.000-04:00,2023-11-09T20:18:40.000-04:00,2000005022926071,paid,42783998936,fulfillment
137,1,MLB3427769549,FULLIZZO6480,2023-11-09T20:26:55.000-04:00,2023-11-09T20:25:54.000-04:00,None,paid,42784023008,fulfillment
138,1,MLB1992889189,FULLLIVERPOOL3NI5BM,2023-11-09T20:27:11.000-04:00,2023-11-09T20:27:10.000-04:00,2000005022956879,paid,42784027932,fulfillment
139,1,MLB1992889189,FULLLIVERPOOL3NI5AM,2023-11-09T20:30:13.000-04:00,2023-11-09T20:30:12.000-04:00,2000005022968751,cancelled,42784038436,fulfillment


## Pegando códigos para comparar os produtos que queremos

In [186]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)
df_codes.shape

C:\Users\couti\AppData\Local\Temp\ipykernel_1304\2578135159.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


(878, 7)

In [187]:
df_codes.columns
df_order_res.columns

Index(['quantity', 'item.id', 'item.seller_sku', 'date_closed',
       'date_approved', 'pack_id', 'status', 'shipping_id', 'logistic_type'],
      dtype='object')

In [188]:
df_codes.head(3)

,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,509517168,7899028808537,JFGN34621,MLB924922735,FULLP10NINJA10FTC,2023-11-10 16:32:30.451360,None
1,598796832,7899028824407,ERIM51807,MLB949788598,P10LNINJA10FT,2023-11-10 16:32:30.451360,None
2,565287467,7899028808773,HVCS52763,MLB950214971,FULLLWNINJA10FT,2023-11-10 16:32:30.451360,None


In [189]:
df_order_res.head(1)

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type
0,1,MLB3427769549,FULLIZZO6480,2023-11-08T20:22:47.000-04:00,2023-11-08T20:22:47.000-04:00,None,paid,42780148489,fulfillment


## Comparando os dois DFs

In [190]:
# df_filtered = df_order_res.merge(df_codes, left_on='item.seller_sku', right_on='ml_sku', how='inner')
df_filtered = pd.merge(df_order_res, df_codes, left_on=['item.id', 'item.seller_sku'], right_on=['ml_code', 'ml_sku'], how='inner')

# df_filtered = df_filtered.drop(['updated_at', 'created_at'], axis=1)
# df_filtered = df_filtered.drop(['item.id', 'updated_at', 'created_at'], axis=1)
df_filtered.head(3)

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,1,MLB1453957901,FULLKALANI15414,2023-11-08T20:33:14.000-04:00,2023-11-08T20:33:14.000-04:00,None,paid,42780312778,fulfillment,512623216,7899574325878,NWXH48678,MLB1453957901,FULLKALANI15414,2023-11-10 16:32:30.451360,None
1,1,MLB1971185030,FULLBRANCO3440,2023-11-08T20:42:51.000-04:00,2023-11-08T20:42:50.000-04:00,2000005018869963,paid,42780347140,fulfillment,741799597,7897626802353,WOJB37093,MLB1971185030,FULLBRANCO3440,2023-11-10 16:32:30.451360,None
2,1,MLB3780803056,FULLBASSOCLA09,2023-11-08T20:46:36.000-04:00,2023-11-08T20:46:35.000-04:00,2000005018888075,paid,42780249069,fulfillment,729129329,7898942136139,NaN,MLB3780803056,FULLBASSOCLA09,2023-11-10 16:32:30.451360,None


In [191]:
print(df_filtered.shape)

(93, 16)


In [192]:
print(df_filtered['quantity'].value_counts(),'\n')

# print(df_filtered['item.seller_sku'].value_counts(),'\n')

print(df_filtered['status'].value_counts(),'\n')

# print(df_filtered['shipping_id'].value_counts(),'\n')

print(df_filtered['logistic_type'].value_counts(),'\n')

# print(df_filtered['tiny_id'].value_counts(),'\n')

# print(df_filtered['ml_inventory_id'].value_counts(),'\n')

print(df_filtered['ml_code'].value_counts(),'\n')


quantity
1    88
2     5
Name: count, dtype: int64 

status
paid         91
cancelled     2
Name: count, dtype: int64 

logistic_type
fulfillment    93
Name: count, dtype: int64 

ml_code
MLB1992889189    16
MLB2770292345     7
MLB2075516288     7
MLB3461864178     6
MLB2001037689     4
MLB1813530858     4
MLB2004757564     4
MLB1942106803     4
MLB1991060554     4
MLB1992567302     3
MLB2606294854     3
MLB2075798323     2
MLB3778529236     2
MLB1813063328     2
MLB1662970458     2
MLB1557018312     2
MLB1629577482     1
MLB1843745999     1
MLB3778540120     1
MLB1453957901     1
MLB2075548286     1
MLB2024002438     1
MLB1400355425     1
MLB2001122140     1
MLB3339515153     1
MLB1403763907     1
MLB2660440797     1
MLB2000305301     1
MLB2216644379     1
MLB1971185030     1
MLB1403791227     1
MLB2171831134     1
MLB1629353758     1
MLB1738241921     1
MLB2003190501     1
MLB3780803056     1
MLB3696483496     1
Name: count, dtype: int64 



In [193]:
# print(df_filtered['tiny_id'].value_counts(),'\n')

In [194]:
# dfx = df_filtered[df_filtered['tiny_id'] == 747848159]
# dfx.head(5)

## Contagem de produtos vendidos

In [195]:
dfx = df_filtered.head()

In [196]:
soma_por_shipping_id = df_filtered.groupby('shipping_id')['quantity'].sum().reset_index()
soma_por_shipping_id

,shipping_id,quantity
0,42780249069,1
1,42780312778,1
2,42780347140,1
3,42780352733,4
4,42780386625,4
...,...,...
70,42783987128,4
71,42783998936,1
72,42784027932,1
73,42784038436,1


In [197]:
# soma_por_tiny_id = df_filtered.groupby('tiny_id')['quantity'].sum().reset_index()
# soma_por_tiny_id

In [198]:
dfx = df_filtered[df_filtered['ml_inventory_id'] == 'BOWK48319']
dfx

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at


In [199]:
x = df_filtered[df_filtered['ml_inventory_id'] == 'NaN']
x.shape

(12, 16)

In [200]:
x

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
2,1,MLB3780803056,FULLBASSOCLA09,2023-11-08T20:46:36.000-04:00,2023-11-08T20:46:35.000-04:00,2000005018888075,paid,42780249069,fulfillment,729129329,7898942136139,NaN,MLB3780803056,FULLBASSOCLA09,2023-11-10 16:32:30.451360,None
30,1,MLB3461864178,FULLP10LNINJA30FT,2023-11-09T06:02:15.000-04:00,2023-11-09T06:02:14.000-04:00,2000005019920985,paid,42781076815,fulfillment,724202643,7899028824551,NaN,MLB3461864178,FULLP10LNINJA30FT,2023-11-10 16:32:30.451360,None
31,1,MLB3461864178,FULLP10LNINJA30FT,2023-11-09T07:26:33.000-04:00,2023-11-09T07:26:33.000-04:00,2000005020051519,paid,42781332622,fulfillment,724202643,7899028824551,NaN,MLB3461864178,FULLP10LNINJA30FT,2023-11-10 16:32:30.451360,None
32,2,MLB3461864178,FULLP10LNINJA30FT,2023-11-09T07:45:14.000-04:00,2023-11-09T07:45:13.000-04:00,2000005020092391,paid,42781263167,fulfillment,724202643,7899028824551,NaN,MLB3461864178,FULLP10LNINJA30FT,2023-11-10 16:32:30.451360,None
33,1,MLB3461864178,FULLP10LNINJA30FT,2023-11-09T12:42:27.000-04:00,2023-11-09T12:42:26.000-04:00,2000005021181955,paid,42782431388,fulfillment,724202643,7899028824551,NaN,MLB3461864178,FULLP10LNINJA30FT,2023-11-10 16:32:30.451360,None
34,1,MLB3461864178,FULLP10LNINJA30FT,2023-11-09T12:49:10.000-04:00,2023-11-09T12:49:10.000-04:00,2000005021204799,paid,42782456764,fulfillment,724202643,7899028824551,NaN,MLB3461864178,FULLP10LNINJA30FT,2023-11-10 16:32:30.451360,None
35,1,MLB3461864178,FULLP10LNINJA30FT,2023-11-09T13:52:44.000-04:00,2023-11-09T13:52:44.000-04:00,2000005021473901,paid,42782695232,fulfillment,724202643,7899028824551,NaN,MLB3461864178,FULLP10LNINJA30FT,2023-11-10 16:32:30.451360,None
42,1,MLB3778529236,FULLBASSOCLA14,2023-11-09T08:48:06.000-04:00,2023-11-09T08:48:06.000-04:00,None,paid,42781558446,fulfillment,729129880,7898529535461,NaN,MLB3778529236,FULLBASSOCLA14,2023-11-10 16:32:30.451360,None
43,1,MLB3778529236,FULLBASSOCLA14,2023-11-09T19:44:51.000-04:00,2023-11-09T19:44:51.000-04:00,None,paid,42783889446,fulfillment,729129880,7898529535461,NaN,MLB3778529236,FULLBASSOCLA14,2023-11-10 16:32:30.451360,None
74,1,MLB3778540120,FULLBASSODE219,2023-11-09T15:09:00.000-04:00,2023-11-09T15:09:00.000-04:00,2000005021782637,paid,42782855675,fulfillment,729130513,7898529534730,NaN,MLB3778540120,FULLBASSODE219,2023-11-10 16:32:30.451360,None


In [201]:
x = soma_por_ml_inventory_id[soma_por_ml_inventory_id['ml_inventory_id'] == 'YXIW04349']
x

,ml_inventory_id,quantity
28,YXIW04349,1


In [202]:
soma_por_ml_inventory_id = df_filtered.groupby('ml_inventory_id')['quantity'].sum().reset_index()
soma_por_ml_inventory_id.head(4)
soma_por_ml_inventory_id

,ml_inventory_id,quantity
0,AVOX05934,1
1,EIPZ51427,8
2,ELQK04205,3
3,FHRY37807,3
4,FSNB97805,4
5,FVTQ20181,1
6,GUSW29256,1
7,HSDO37848,3
8,HTEZ21817,1
9,JDTL81315,4


## Calculando estoque

### Pegando estoque do DB

In [203]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM fulfillment_stock"

df_stock = pd.read_sql(sql_query, conn)

df_stock = df_stock.sort_values(by='created_at', ascending=False)
df_stock['data'] = df_stock['created_at'].dt.date


C:\Users\couti\AppData\Local\Temp\ipykernel_1304\1772980314.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [204]:
df_stock

,ml_inventory_id,available_quantity,created_at,data
19071,UCQQ19395,0,2023-11-10 20:22:52.890804,2023-11-10
18679,LDBL69476,0,2023-11-10 20:22:52.890804,2023-11-10
18677,EDGX70669,0,2023-11-10 20:22:52.890804,2023-11-10
18676,VFWO70372,0,2023-11-10 20:22:52.890804,2023-11-10
18675,GLMK71067,0,2023-11-10 20:22:52.890804,2023-11-10
...,...,...,...,...
18279,EDHV75882,0,2023-10-10 01:00:00.000000,2023-10-10
18278,IVOT73839,0,2023-10-10 01:00:00.000000,2023-10-10
18277,WPGI76636,0,2023-10-10 01:00:00.000000,2023-10-10
18276,UCDR32919,14,2023-10-10 01:00:00.000000,2023-10-10


In [205]:
soma_por_ml_inventory_id.head(1)

,ml_inventory_id,quantity
0,AVOX05934,1


In [223]:
df_orders_f = df_filtered.copy()
df_orders_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   quantity         93 non-null     int64         
 1   item.id          93 non-null     object        
 2   item.seller_sku  93 non-null     object        
 3   date_closed      93 non-null     object        
 4   date_approved    93 non-null     object        
 5   pack_id          47 non-null     object        
 6   status           93 non-null     object        
 7   shipping_id      93 non-null     int64         
 8   logistic_type    93 non-null     object        
 9   tiny_id          93 non-null     int64         
 10  tiny_sku         93 non-null     object        
 11  ml_inventory_id  93 non-null     object        
 12  ml_code          93 non-null     object        
 13  ml_sku           93 non-null     object        
 14  created_at       93 non-null     datetime64[

In [224]:
df_orders_f['date_approved'] = pd.to_datetime(df_orders_f['date_approved'])
df_orders_f['data'] = df_orders_f['date_approved'].dt.date
df_orders_f = df_orders_f.drop(['date_closed', 'created_at', 'updated_at'], axis=1)


In [225]:
df_orders_f.head(1)

,quantity,item.id,item.seller_sku,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,data
0,1,MLB1453957901,FULLKALANI15414,2023-11-08 20:33:14-04:00,None,paid,42780312778,fulfillment,512623216,7899574325878,NWXH48678,MLB1453957901,FULLKALANI15414,2023-11-08


In [226]:
df_stock.head(1)

,ml_inventory_id,available_quantity,created_at,data,has_stock
19071,UCQQ19395,0,2023-11-10 20:22:52.890804,2023-11-10,False


### Verificando se existe no estoque

In [227]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock.sort_values(by='data', ascending=False)
df_stock.head(4)

,ml_inventory_id,available_quantity,created_at,data,has_stock
19071,UCQQ19395,0,2023-11-10 20:22:52.890804,2023-11-10,False
18679,LDBL69476,0,2023-11-10 20:22:52.890804,2023-11-10,False
18677,EDGX70669,0,2023-11-10 20:22:52.890804,2023-11-10,False
18676,VFWO70372,0,2023-11-10 20:22:52.890804,2023-11-10,False


In [228]:

def filtrar_por_periodo(df, data_inicio, data_fim):
    return df[(df['data'] >= data_inicio) & (df['data'] <= data_fim)]

data_inicio = datetime(2023, 11, 9).date()  
data_fim = datetime(2023, 11, 10).date()     

# Filtrar DataFrames com base nas datas definidas
df_stock_filtrado = filtrar_por_periodo(df_stock, data_inicio, data_fim)
df_orders_f_filtrado = filtrar_por_periodo(df_orders_f, data_inicio, data_fim)


df_stock_filtrado

,ml_inventory_id,available_quantity,created_at,data,has_stock
19071,UCQQ19395,0,2023-11-10 20:22:52.890804,2023-11-10,False
18679,LDBL69476,0,2023-11-10 20:22:52.890804,2023-11-10,False
18677,EDGX70669,0,2023-11-10 20:22:52.890804,2023-11-10,False
18676,VFWO70372,0,2023-11-10 20:22:52.890804,2023-11-10,False
18675,GLMK71067,0,2023-11-10 20:22:52.890804,2023-11-10,False
...,...,...,...,...,...
396,UCDR32919,14,2023-11-09 16:33:03.451375,2023-11-09,True
395,GUSW29256,3,2023-11-09 16:33:03.451375,2023-11-09,True
394,EDHV29406,0,2023-11-09 16:33:03.451375,2023-11-09,False
393,ZDLT16176,3,2023-11-09 16:33:03.451375,2023-11-09,True


In [231]:
x = df_stock_filtrado[df_stock_filtrado['ml_inventory_id']=='EDHV29406']
x

,ml_inventory_id,available_quantity,created_at,data,has_stock
18870,EDHV29406,0,2023-11-10 20:22:52.890804,2023-11-10,False
394,EDHV29406,0,2023-11-09 16:33:03.451375,2023-11-09,False


In [229]:


# Contar quantas vezes 'True' aparece para cada 'ml_inventory_id'
contagem_por_inventory_id = df_stock_filtrado[df_stock_filtrado['has_stock']].groupby('ml_inventory_id').size()

# Exibir o resultado
contagem_por_inventory_id

ml_inventory_id
AGDL67534    2
AIRQ38676    2
AJZG23550    2
AVOX05934    2
AWAZ99732    2
            ..
ZXFT21237    2
ZXJO91694    2
ZZGN21450    2
ZZQL21216    2
ZZRV30730    2
Length: 257, dtype: int64

In [234]:
valor_procurado = 'EDHV29406'

quantidade_para_valor_procurado = contagem_por_inventory_id.get(valor_procurado, 0)
print(f"A quantidade para '{valor_procurado}' é: {quantidade_para_valor_procurado}")



A quantidade para 'EDHV29406' é: 0


In [236]:
# Contar quantas vezes 'True' aparece para cada 'ml_inventory_id'
contagem_por_inventory_id = df_stock_filtrado[df_stock_filtrado['has_stock']].groupby('ml_inventory_id').size()

# Converter a Série em uma lista
lista_de_contagens = contagem_por_inventory_id.reset_index(name='contagem').values.tolist()

# Exibir a lista de contagens
print(lista_de_contagens)

[['AGDL67534', 2], ['AIRQ38676', 2], ['AJZG23550', 2], ['AVOX05934', 2], ['AWAZ99732', 2], ['AZVM53003', 2], ['AZZD53396', 2], ['BOWK32167', 2], ['BOWK48319', 2], ['BOZK72331', 2], ['BPGQ06190', 2], ['BPGQ10723', 2], ['BUIO60614', 2], ['BXUC56742', 2], ['BZEY80870', 2], ['CCQW73335', 2], ['CDBO10673', 2], ['CDBO11155', 2], ['CIKE94068', 2], ['CJBY66878', 2], ['CLMF99895', 2], ['CMXV62869', 2], ['CNHG66088', 2], ['CRMK90073', 2], ['DBBD03410', 2], ['DCGQ72320', 2], ['DCPP20168', 2], ['DGCL82617', 2], ['DGLC52390', 1], ['DHEV27121', 2], ['DIAC90428', 2], ['DJDL49028', 2], ['DODL05588', 2], ['DSGP06967', 2], ['DSGP06979', 2], ['DWNN86592', 2], ['DXJZ75064', 2], ['DZVJ05686', 1], ['EALS35835', 2], ['EATC35944', 2], ['EIPZ51427', 2], ['ELKB12099', 2], ['ELKX63703', 2], ['ELQK04205', 2], ['ELUM57266', 2], ['ENML56386', 2], ['ERIM51807', 2], ['ESVG06709', 2], ['FFIA05428', 2], ['FFTO88214', 2], ['FHRY17474', 2], ['FHRY37807', 2], ['FRBW82060', 2], ['FSNB97805', 2], ['FTGG16520', 2], ['FVTQ201

In [241]:
# Extrair o primeiro elemento de cada sublista
ml_inventory_ids = [item[0] for item in lista_de_contagens]

# Exibir os ml_inventory_ids
print(ml_inventory_ids)
print(len(ml_inventory_ids))

['AGDL67534', 'AIRQ38676', 'AJZG23550', 'AVOX05934', 'AWAZ99732', 'AZVM53003', 'AZZD53396', 'BOWK32167', 'BOWK48319', 'BOZK72331', 'BPGQ06190', 'BPGQ10723', 'BUIO60614', 'BXUC56742', 'BZEY80870', 'CCQW73335', 'CDBO10673', 'CDBO11155', 'CIKE94068', 'CJBY66878', 'CLMF99895', 'CMXV62869', 'CNHG66088', 'CRMK90073', 'DBBD03410', 'DCGQ72320', 'DCPP20168', 'DGCL82617', 'DGLC52390', 'DHEV27121', 'DIAC90428', 'DJDL49028', 'DODL05588', 'DSGP06967', 'DSGP06979', 'DWNN86592', 'DXJZ75064', 'DZVJ05686', 'EALS35835', 'EATC35944', 'EIPZ51427', 'ELKB12099', 'ELKX63703', 'ELQK04205', 'ELUM57266', 'ENML56386', 'ERIM51807', 'ESVG06709', 'FFIA05428', 'FFTO88214', 'FHRY17474', 'FHRY37807', 'FRBW82060', 'FSNB97805', 'FTGG16520', 'FVTQ20181', 'FZQP10396', 'GCYF86172', 'GHFV89567', 'GLMK21133', 'GLMK66383', 'GMII66042', 'GPGU38258', 'GUSW29256', 'HFEB10246', 'HGRL10383', 'HHXZ78268', 'HNWI37644', 'HSDO10298', 'HSDO37848', 'HTEZ21817', 'HTNQ39602', 'HVCS52763', 'ITDN80392', 'ITMG31457', 'ITND36634', 'ITQQ97036'

In [240]:
df_orders_f_filtrado.shape

(72, 14)

In [243]:
df_orders_f_filtrado

,quantity,item.id,item.seller_sku,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,data
13,1,MLB1992889189,FULLLIVERPOOL3NI5AM,2023-11-09 20:30:12-04:00,2000005022968751,cancelled,42784038436,fulfillment,698412654,7897937421014,FSNB97805,MLB1992889189,FULLLIVERPOOL3NI5AM,2023-11-09
16,1,MLB2770292345,FULLVANDOREN2SR2115,2023-11-09 17:18:36-04:00,None,paid,42783298415,fulfillment,748837437,7897626822108,EIPZ51427,MLB2770292345,FULLVANDOREN2SR2115,2023-11-09
17,2,MLB2770292345,FULLVANDOREN2SR2115,2023-11-09 18:23:27-04:00,2000005022511791,paid,42783622686,fulfillment,748837437,7897626822108,EIPZ51427,MLB2770292345,FULLVANDOREN2SR2115,2023-11-09
19,1,MLB1992567302,FULLNI5AM,2023-11-09 13:40:45-04:00,2000005021427209,paid,42782536547,fulfillment,698412654,7897937421014,JJSZ06277,MLB1992567302,FULLNI5AM,2023-11-09
20,1,MLB1992567302,FULLNI5AM,2023-11-09 19:01:57-04:00,2000005022647939,paid,42783746346,fulfillment,698412654,7897937421014,JJSZ06277,MLB1992567302,FULLNI5AM,2023-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09 20:15:01-04:00,2000005022913903,paid,42783987128,fulfillment,729129899,7898529535546,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09
89,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09 20:15:01-04:00,2000005022913903,paid,42783987128,fulfillment,565665071,Palheta,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09
90,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09 20:15:01-04:00,2000005022913903,paid,42783987128,fulfillment,565663877,PPalheta,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09
91,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09 20:15:01-04:00,2000005022913903,paid,42783987128,fulfillment,746249012,RH-01SV,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-09


In [245]:
# Calcular a soma de 'quantity' para cada 'item.id'
soma_por_item_id = df_orders_f_filtrado.groupby('item.id')['quantity'].sum()

# Exibir o resultado
print(soma_por_item_id)


item.id
MLB1400355425     1
MLB1403763907     1
MLB1403791227     1
MLB1557018312     2
MLB1629353758     2
MLB1629577482     1
MLB1662970458     2
MLB1813063328     2
MLB1843745999     1
MLB1942106803     4
MLB1991060554     5
MLB1992567302     2
MLB1992889189    12
MLB2000305301     1
MLB2001037689     4
MLB2001122140     1
MLB2024002438     1
MLB2075516288     7
MLB2075548286     1
MLB2075798323     2
MLB2171831134     1
MLB2216644379     1
MLB2606294854     2
MLB2660440797     1
MLB2770292345     6
MLB3339515153     1
MLB3461864178     7
MLB3696483496     1
MLB3778529236     2
MLB3778540120     1
Name: quantity, dtype: int64


In [ ]:
import pandas as pd

# Exemplo de DataFrames (substitua pelos seus DataFrames reais)
data_stock = {
    'data': ['2023-10-01', '2023-10-01', '2023-10-02', '2023-10-02', '2023-10-02'],
    'ml_inventory_id': ['item1', 'item2', 'item1', 'item2', 'item3'],
    'has_stock': [True, False, True, True, False],
}

data_orders = {
    'data': ['2023-10-01', '2023-10-01', '2023-10-02', '2023-10-02', '2023-10-02'],
    'item.id': ['item1', 'item2', 'item1', 'item2', 'item3'],
    'quantity': [10, 5, 8, 3, 6],
}

df_stock_filtrado = pd.DataFrame(data_stock)
df_orders_f_filtrado = pd.DataFrame(data_orders)

# Calcular a soma de 'quantity' para cada 'item.id' em df_orders_f_filtrado
soma_por_item_id = df_orders_f_filtrado.groupby('item.id')['quantity'].sum().reset_index(name='soma_quantity')

# Mesclar df_stock_filtrado com a soma_por_item_id usando 'ml_inventory_id' e 'item.id'
df_stock_filtrado = pd.merge(df_stock_filtrado, soma_por_item_id, left_on='ml_inventory_id', right_on='item.id', how='left')

# Preencher valores NaN com 0
df_stock_filtrado['soma_quantity'] = df_stock_filtrado['soma_quantity'].fillna(0)

# Exibir o DataFrame resultante
print(df_stock_filtrado)
